In [51]:
import numpy as np
from os import scandir
import pandas as pd
import PyPDF2
import re
import pprint as pp

In [52]:
def get_statement_transactions(file: str, regex: re.Pattern) -> list[str]:
    text = ''
    with open(file, "rb") as fn:
        reader = PyPDF2.PdfReader(fn)
        for page in reader.pages:
            text += page.extract_text()

    return re.findall(regex, text)

In [53]:
PNC_REGEX = re.compile(r"\d{2}/\d{2} \d*,?\d*\.\d{2} .*(?=\n\d{2}/\d{2}|\nOnline and Electronic|\nBanking/Debit Card|\n4Virtual Wallet|\nDaily Balance|\nOther Deductions)|\d{2}/\d{2} \d*,?\d*\.\d{2} .*\n.*", re.MULTILINE)
FOLDER_PATH = "..\data\PNC"

In [54]:
raw_data = []
for stmt in scandir(FOLDER_PATH):
    raw_data += get_statement_transactions(stmt, PNC_REGEX)

In [55]:
clean_data = []
for line in raw_data:
    clean_data.append(line.replace("\n", " ").strip())

In [69]:
before_df = []
for line in clean_data:
    date, amount, *recip = line.split()
    amount = amount.replace(',', '')
    if amount.startswith('.'):
        amount = '0' + amount
    recip = ' '.join(recip)
    before_df += [[date, 'Checking', amount, recip, "", "", "", ""]]

In [71]:
df = pd.DataFrame(before_df, columns=['Date', 'Account', 'Amount', 'Recipient', 'Category', 'Sub_Category', 'Project', 'Note'])


In [72]:
df

,Date,Account,Amount,Recipient,Category,Sub_Category,Project,Note
0,03/05,Checking,200.00,Direct Deposit - Online PNC Bank XXXXXX4834,,,,
1,03/06,Checking,264.02,Mobile Deposit Reference No. 072420331,,,,
2,03/18,Checking,919.86,Mobile Deposit Reference No. 077504534,,,,
3,03/25,Checking,394.00,Direct Deposit - Cashout Venmo XXXXXX4841,,,,
4,03/20,Checking,99.98,POS Purchase Burlington Sto Clifton NJ,,,,
...,...,...,...,...,...,...,...,...
894,10/03,Checking,1564.00,Web Pmt- Payment Venmo 1022642658191,,,,
895,10/06,Checking,3.62,Web Pmt- Inst Xfer Paypal Ebayincship,,,,
896,10/11,Checking,4.16,Web Pmt- Inst Xfer Paypal Ebayincship,,,,
897,10/20,Checking,211.22,Web Pmt- Epay Chase Credit Crd 6269237537,,,,
